<a href="https://colab.research.google.com/github/danielelbrecht/CAP5610-HW-2/blob/master/CAP5610_HW2_model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model # 3:  Add in average pooling, elu activations

In [1]:
from keras.datasets import cifar10
import numpy as np
import sklearn
import tensorflow as tf
from tensorflow.keras import layers, utils

Using TensorFlow backend.


In [0]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [0]:
# Preprocess data
train_images = train_images.reshape((50000, 32, 32, 3)) / 255.0
test_images = test_images.reshape((10000, 32, 32, 3)) / 255.0

train_labels_categorical = utils.to_categorical(train_labels, num_classes=10, dtype='float32')
test_labels_categorical = utils.to_categorical(test_labels, num_classes=10, dtype='float32')

# Implement simple holdout validation
length = len(train_images)
threshold = int(length * 0.8)

train_set_images = train_images[0:threshold]
train_set_labels = train_labels_categorical[0:threshold]

val_set_images = train_images[threshold:length]
val_set_labels = train_labels_categorical[threshold:length]

In [7]:
#Define model

model = tf.keras.Sequential()

# First convolutional module
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='elu', input_shape=(32,32,3)))
model.add(layers.AveragePooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.2))

# Second convolutional module
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation='elu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.2))


# Fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='elu'))
model.add(layers.Dense(64, activation='elu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
average_pooling2d (AveragePo (None, 15, 15, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 6, 6, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
__________

In [8]:
# Train model
history = model.fit(train_set_images, 
                      train_set_labels, 
                      epochs=10,
                      validation_data=(val_set_images, val_set_labels))

Train on 40000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
40000/40000 [==============================] - 312s 8ms/sample - loss: 1.4672 - acc: 0.4790 - val_loss: 1.2814 - val_acc: 0.5528
Epoch 2/10
40000/40000 [==============================] - 306s 8ms/sample - loss: 1.1698 - acc: 0.5900 - val_loss: 1.1799 - val_acc: 0.5911
Epoch 3/10
40000/40000 [==============================] - 306s 8ms/sample - loss: 1.0442 - acc: 0.6338 - val_loss: 1.1018 - val_acc: 0.6212
Epoch 4/10
40000/40000 [==============================] - 303s 8ms/sample - loss: 0.9465 - acc: 0.6690 - val_loss: 1.0643 - val_acc: 0.6410
Epoch 5/10
40000/40000 [==============================] - 304s 8ms/sample - loss: 0.8703 - acc: 0.6976 - val_loss: 1.2091 - val_acc: 0.6103
Epoch 6/10
40000/40000 [==============================] - 159s 4ms/sample - loss: 0.8062 - acc: 0.7169 - val_loss: 1.0975 - val_acc: 0.6366
Epoch 7/10
40000/40000 [==============================] - 155s